In [1]:
import openai
from pathlib import Path

APIキーは以下のURLから取得できます。  
https://platform.openai.com/account/api-keys  

【注意】  
<font color="Crimson">APIキーが流出すると他人に利用されて料金を請求されます！</font>  
絶対に共有してはいけません。  

In [2]:
# APIキーの読み込み
# キーを格納したテキストファイルを指定して読み込み
with open(f'{Path.home()}/.private_info/api_key.txt') as f:
    openai.api_key = f.read()

# パラメータ設定

- temperature (optional): float, 0~2, defaults to 1 
  
サンプリング温度(※)を 0〜2 の間で指定します。
値が低いほど、より関連性の高い単語が選ばれやすくなり、値が高いほど、より多様な単語が選ばれやすくなります。  
この値か top_p パラメーター を変更することを推奨していますが、両方は同時に変更することは推奨されません。  
  
※サンプリング温度について  
文章生成時に使用される確率分布の調整パラメーターの一つで、生成される文章の多様性や不確実性を調整するために使用されます。  
サンプリング温度が低い場合、より一貫性がある文章が生成され、単語の組み合わせの変化が少なくなります。  
サンプリング温度が高い場合、より多様な文章が生成され、単語の組み合わせがより多様になります。  
ChatGPT のデフォルトは 0.7 です。  
  
- top_p (optional): float, 0~1, optional, defaults to 1
  
サンプリング温度に代わるものとして利用されます。  
値が小さいほど、生成される文章は文法的により正確になります。  
値が大きいほど、より多様な単語が選択されるため、生成される文章はより多様になります。  
tenperature パラメーターとの併用は推奨されていません。  
ChatGPT のデフォルトは 0.9~0.95 です。  
  
- その他のパラメータは[公式ドキュメント](https://platform.openai.com/docs/api-reference/models)を参照  
  
[日本語参考サイト3](https://ai-create.net/magazine/2023/03/02/post-10647/)  

# code_generator  
  
「Pythonでやりたいこと」を関数に入力

In [4]:
def code_generator(message: str):
    # system role
    role_content = 'Create Python scripts. Then, your answer must be the scripts only.'
    messages = [
        {'role': 'system', 'content': role_content},
    ]
    # input message
    messages.append(
        {'role': 'user', 'content': message},
    )
    # get reply
    chat_completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        # temperature=0.1,  # Only one of "temperature" and "top_p" should be modified.
        top_p=0.1,  # Only one of "temperature" and "top_p" should be modified.
    )
    reply = chat_completion.choices[0].message.content
    print(reply)

In [5]:
code_generator('10以下のフィボナッチ数列をリストで出力する')

```
fib = [0, 1]
while fib[-1] < 10:
    fib.append(fib[-1] + fib[-2])
print(fib[:-1])
```


In [6]:
fib = [0, 1]
while fib[-1] < 10:
    fib.append(fib[-1] + fib[-2])
print(fib[:-1])

[0, 1, 1, 2, 3, 5, 8]


In [7]:
code_generator('10以下のフィボナッチ数列をリストで出力する。このとき0はリストに含まない。')

```
fibonacci = [1, 1]
while fibonacci[-1] < 10:
    fibonacci.append(fibonacci[-1] + fibonacci[-2])
print(fibonacci[:-1])
```


In [8]:
fib = [1, 1]
while fib[-1] < 10:
    fib.append(fib[-1] + fib[-2])
print(fib[:-1])

[1, 1, 2, 3, 5, 8]


# code_fixer  
  
「うまくいかないコード」を関数に入力

In [9]:
def code_fixer(message: str):
    # system role
    role_content = 'Fix bugs in the Python scripts. Then, your answer must be the scripts only.'
    messages = [
        {'role': 'system', 'content': role_content},
    ]
    # input message
    messages.append(
        {'role': 'user', 'content': message},
    )
    # get reply
    chat_completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        # temperature=0.1,  # Only one of "temperature" and "top_p" should be modified.
        top_p=0.1,  # Only one of "temperature" and "top_p" should be modified.
    )
    reply = chat_completion.choices[0].message.content
    print(reply)

In [10]:
code_fixer(
"""
import pandas as pd

df = pd.read_csv('hogefuga.excel')
"""
)

import pandas as pd

df = pd.read_excel('hogefuga.xlsx') # corrected file extension from .excel to .xlsx


In [10]:
code_fixer(
"""
columns_price = ['a', 'b', 'c']
df[columns_price] = df['average'] - df[columns_price]
"""
)

There are a few issues with this script. Here's the corrected version:

```
columns_price = ['a', 'b', 'c']
df[columns_price] = df[columns_price].sub(df['average'], axis=0)
```

Explanation:

1. We need to use the `sub` method to subtract the `average` column from the `columns_price` columns. 
2. We need to specify `axis=0` to ensure that the subtraction is done row-wise. 
3. We need to use `df[columns_price]` on both sides of the equation to ensure that we are only modifying the `columns_price` columns and not creating new columns.


# code_generator2  
  
交互にやり取りできるようにしました。  

In [11]:
class CodeGenerator2:

    def __init__(self, api_key):
        openai.api_key = api_key
        self.role = 'Create Python scripts. Then, your answer must be the scripts only.'
        self.messages = [
            {'role': 'system', 'content': self.role},
        ]

    def generate(self, message: str):
        # input message
        self.messages.append(
            {'role': 'user', 'content': message},
        )
        # get reply
        chat_completion = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=self.messages,
            # temperature=0.1,  # Only one of "temperature" and "top_p" should be modified.
            top_p=0.1,  # Only one of "temperature" and "top_p" should be modified.
        )
        reply = chat_completion.choices[0].message.content
        print(reply)
        self.messages.append(
            {'role': 'assistant', 'content': reply}
        )

    def clear(self):
        self.messages = [
            {'role': 'system', 'content': self.role},
        ]

In [12]:
with open(f'{Path.home()}/.private_info/api_key.txt') as f:
    api_key = f.read()

cg = CodeGenerator2(api_key=api_key)

In [13]:
# 最初の質問
cg.generate('10以下のフィボナッチ数列をリストで出力する')

```
fib = [0, 1]
while fib[-1] < 10:
    fib.append(fib[-1] + fib[-2])
print(fib[:-1])
```


In [14]:
# 続けて質問
cg.generate('0は出力のリストに含まないように修正して')

```
fib = [1, 1]
while fib[-1] < 10:
    fib.append(fib[-1] + fib[-2])
print(fib[:-1])
```


In [15]:
# 過去のやり取りはスタックされている
cg.messages

[{'role': 'system',
  'content': 'Create Python scripts. Then, your answer must be the scripts only.'},
 {'role': 'user', 'content': '10以下のフィボナッチ数列をリストで出力する'},
 {'role': 'assistant',
  'content': '```\nfib = [0, 1]\nwhile fib[-1] < 10:\n    fib.append(fib[-1] + fib[-2])\nprint(fib[:-1])\n```'},
 {'role': 'user', 'content': '0は出力のリストに含まないように修正して'},
 {'role': 'assistant',
  'content': '```\nfib = [1, 1]\nwhile fib[-1] < 10:\n    fib.append(fib[-1] + fib[-2])\nprint(fib[:-1])\n```'}]

In [16]:
# 解決したのでリセット
cg.clear()
cg.messages

[{'role': 'system',
  'content': 'Create Python scripts. Then, your answer must be the scripts only.'}]

In [17]:
# リセットしたら過去の文脈がなくなるのでChatGPT君が困惑する
cg.generate('0は出力のリストに含まないように修正して')

以下は、リスト内包表記を使用して、0を含まないリストを作成するPythonスクリプトです。

```python
original_list = [1, 2, 0, 3, 0, 4, 5, 0, 6]
new_list = [x for x in original_list if x != 0]
print(new_list)
```

出力：

```
[1, 2, 3, 4, 5, 6]
```
